In [53]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

In [54]:
import numpy as np
import pandas as pd

In [55]:
embedding_data = pd.read_pickle('./df_video_info.pkl')
feature_data = pd.read_feather('./sponsorTimes_filtered_videoID_level.feather')
data = pd.read_pickle('./df_video_info_with_features.pkl')

In [56]:
# pd.merge(embedding_data,feature_data, on = 'videoID')
data.columns

Index(['videoID', 'channelID', 'title', 'name', 'fe_segment_duration_0',
       'fe_segment_duration_1', 'fe_segment_duration_2', 'views_0', 'views_1',
       'views_2',
       ...
       'c374', 'c375', 'c376', 'c377', 'c378', 'c379', 'c380', 'c381', 'c382',
       'c383'],
      dtype='object', length=402)

In [57]:
feature_data.columns, feature_data.shape

(Index(['videoID', 'fe_segment_duration_0', 'fe_segment_duration_1',
        'fe_segment_duration_2', 'views_0', 'views_1', 'views_2', 'votes_0',
        'votes_1', 'votes_2', 'videoDuration',
        'fe_ratio_segment_duration_with_total_video_duration_0',
        'fe_ratio_segment_duration_with_total_video_duration_1',
        'fe_ratio_segment_duration_with_total_video_duration_2',
        'fe_more_than_35_perc_sponsorship_content'],
       dtype='object'),
 (2363, 15))

In [58]:
feature_data

# videoID, views_segment1, views_segment2, views_segment3, views_segment4, channelID, title, embbedding, fe_segment1_duration,
# fe_segment2_duration, fe_segment3_duration, fe_segment4_duration, fe_ratio_segment1_duration_with_total_video_duration, 
# fe_ratio_segment2_duration_with_total_video_duration, fe_ratio_segment3_duration_with_total_video_duration,
# fe_ratio_segment4_duration_with_total_video_duration, fe_more_than_30_perc_sponsorship_content, fe_count_sponsorships

# videoID, ratios, embedding, 

,videoID,fe_segment_duration_0,fe_segment_duration_1,fe_segment_duration_2,views_0,views_1,views_2,votes_0,votes_1,votes_2,videoDuration,fe_ratio_segment_duration_with_total_video_duration_0,fe_ratio_segment_duration_with_total_video_duration_1,fe_ratio_segment_duration_with_total_video_duration_2,fe_more_than_35_perc_sponsorship_content
0,--kZomtrtIQ,5.756,0.000,0.000,0,0,0,0,0,0,465.046,0.012377,0.000000,0.000000,0
1,-2MyBawvlts,5.987,0.000,0.000,1,0,0,0,0,0,636.064,0.009413,0.000000,0.000000,0
2,-3AfFa0rV6Q,24.202,22.070,41.269,0,0,0,0,0,0,2477.000,0.009771,0.008910,0.016661,0
3,-3Q-k4WQTDI,7.384,0.000,0.000,0,0,0,0,0,0,242.024,0.030509,0.000000,0.000000,0
4,-6WBAaHqT8g,5.334,0.000,0.000,0,0,0,0,0,0,242.024,0.022039,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,zwLHJlNMlf4,0.000,24.599,0.000,0,76,0,0,0,0,1092.000,0.000000,0.022527,0.000000,0
2359,zxKURXHy6es,46.073,0.000,0.000,0,0,0,0,0,0,733.073,0.062849,0.000000,0.000000,0
2360,zxdOcHOrAdE,0.000,20.980,0.000,0,0,0,0,0,0,61.060,0.000000,0.343596,0.000000,0
2361,zxi7Rm-lWTg,43.520,0.000,43.337,36,0,11,1,0,0,555.000,0.078414,0.000000,0.078085,0


In [838]:
feature_data[feature_data.videoID == '2USGSuPe8SQ'].head(1).T

,0
videoID,2USGSuPe8SQ
startTime,0.0
endTime,0.0
votes,0
locked,0
timeSubmitted,1634524475139
views,119
category,intro
actionType,skip
videoDuration,1800.081


In [833]:
embeddings_expanded = pd.DataFrame(np.array(feature_data['embbedding'].apply(lambda row: row.strip('[').strip(']').strip(' ').replace('\n', '').replace('  ', ' ').replace(' ', ',').replace('[', '').replace(']', '').split(','))).tolist()).astype('float64')
embeddings_expanded.columns = embeddings_expanded.columns.astype(str)
embeddings_expanded.shape

(4209, 384)

In [834]:
data = feature_data.join(embeddings_expanded)
print(np.isinf(data.fe_ratio_segment_duration_with_total_video_duration).values.sum()) # get number of inifinity values
data.replace([np.inf, -np.inf], np.nan, inplace = True)
data.shape

39


(4209, 406)

In [59]:
# to discuss: Embedding 384th value is coming out as NaN in a lot of cases
X_pd = data.drop(['videoID',  'channelID', 'title', 'name'], axis = 1)
print(X_pd.isnull().sum().sum())
X_pd.dropna(axis = 0, inplace = True)
X_pd.shape

0


(2363, 398)

In [60]:
X_pd.describe()

,fe_segment_duration_0,fe_segment_duration_1,fe_segment_duration_2,views_0,views_1,views_2,votes_0,votes_1,votes_2,videoDuration,...,c374,c375,c376,c377,c378,c379,c380,c381,c382,c383
count,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,...,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000
mean,20.137511,27.236475,8.203986,14.888701,53.520948,5.762167,0.019467,-0.002539,0.003386,951.087211,...,0.008813,0.000777,-0.026745,0.025628,-0.035646,-0.007002,0.054156,-0.058837,-0.036512,0.027963
std,42.110801,69.460023,27.732029,75.225058,318.610721,44.939315,0.210946,0.442256,0.126794,797.083870,...,0.040686,0.048003,0.046151,0.040245,0.049826,0.042586,0.049997,0.056217,0.050072,0.042365
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-15.000000,-2.000000,12.011000,...,-0.118398,-0.160237,-0.160714,-0.109925,-0.185898,-0.139312,-0.107283,-0.209248,-0.175477,-0.110835
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,447.362500,...,-0.019731,-0.030952,-0.059308,-0.001248,-0.067613,-0.037168,0.020090,-0.100207,-0.072208,-0.001044
50%,10.074000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,832.000000,...,0.008296,0.000944,-0.028553,0.024700,-0.035831,-0.009026,0.054938,-0.059038,-0.036842,0.028672
75%,24.063600,26.523500,0.000000,3.000000,4.500000,0.000000,0.000000,0.000000,0.000000,1185.750500,...,0.035463,0.034653,0.004482,0.051728,-0.002209,0.021885,0.088153,-0.018914,-0.002891,0.057265
max,1114.389000,1101.220000,499.816000,2358.000000,13089.000000,1431.000000,4.000000,3.000000,2.000000,7811.000000,...,0.148813,0.126042,0.124663,0.171951,0.135764,0.130991,0.215836,0.159757,0.138907,0.170253


In [61]:
X_pd.std().sort_values()


c127                     0.000000e+00
c223                     0.000000e+00
c319                     3.925272e-09
c157                     2.230743e-02
c338                     3.083827e-02
                             ...     
views_2                  4.493932e+01
fe_segment_duration_1    6.946002e+01
views_0                  7.522506e+01
views_1                  3.186107e+02
videoDuration            7.970839e+02
Length: 398, dtype: float64

In [62]:
norm_features = ['videoDuration', 'views_1']

In [63]:
#normalisation of fe_submission_time -> has high standard deviation
for col in norm_features:
    X_pd[col] = (X_pd[col] - X_pd[col].min()) / (X_pd[col].max() - X_pd[col].min() )

# standardization
# X_pd['fe_submission_time'] = (X_pd['fe_submission_time'] - X_pd['fe_submission_time'].mean()) / X_pd['fe_submission_time'].std()
# X_pd['fe_segment_duration'] = (X_pd['fe_segment_duration'] - X_pd['fe_segment_duration'].mean()) / X_pd['fe_segment_duration'].std()


In [64]:
X_pd.describe()

,fe_segment_duration_0,fe_segment_duration_1,fe_segment_duration_2,views_0,views_1,views_2,votes_0,votes_1,votes_2,videoDuration,...,c374,c375,c376,c377,c378,c379,c380,c381,c382,c383
count,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,...,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000,2363.000000
mean,20.137511,27.236475,8.203986,14.888701,0.004089,5.762167,0.019467,-0.002539,0.003386,0.120410,...,0.008813,0.000777,-0.026745,0.025628,-0.035646,-0.007002,0.054156,-0.058837,-0.036512,0.027963
std,42.110801,69.460023,27.732029,75.225058,0.024342,44.939315,0.210946,0.442256,0.126794,0.102203,...,0.040686,0.048003,0.046151,0.040245,0.049826,0.042586,0.049997,0.056217,0.050072,0.042365
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-15.000000,-2.000000,0.000000,...,-0.118398,-0.160237,-0.160714,-0.109925,-0.185898,-0.139312,-0.107283,-0.209248,-0.175477,-0.110835
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055822,...,-0.019731,-0.030952,-0.059308,-0.001248,-0.067613,-0.037168,0.020090,-0.100207,-0.072208,-0.001044
50%,10.074000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105140,...,0.008296,0.000944,-0.028553,0.024700,-0.035831,-0.009026,0.054938,-0.059038,-0.036842,0.028672
75%,24.063600,26.523500,0.000000,3.000000,0.000344,0.000000,0.000000,0.000000,0.000000,0.150499,...,0.035463,0.034653,0.004482,0.051728,-0.002209,0.021885,0.088153,-0.018914,-0.002891,0.057265
max,1114.389000,1101.220000,499.816000,2358.000000,1.000000,1431.000000,4.000000,3.000000,2.000000,1.000000,...,0.148813,0.126042,0.124663,0.171951,0.135764,0.130991,0.215836,0.159757,0.138907,0.170253


In [78]:
# Xtrain, Xtest = train_test_split(X, train_size = 0.8)
Xtrain_pd, Xtest_pd  = train_test_split(X_pd, train_size = 0.99) # not really needed now - test data not needed now
Xtrain_pd.shape, Xtest_pd.shape

((2339, 398), (24, 398))

In [79]:
#KMeans
kmeans_algo = KMeans(n_clusters = 3)
kmeans_algo.fit(Xtrain_pd)

/Users/apramod/.pyenv/versions/3.8.12/envs/sebicorps/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3)

In [80]:
labels_knn = kmeans_algo.labels_.reshape(len(Xtrain_pd),1)

Xtrain_labelled_knn_pd = pd.DataFrame(Xtrain_pd)
Xtrain_labelled_knn_pd.insert(Xtrain_labelled_knn_pd.shape[1], 'label', labels_knn)

In [81]:
data_with_knnlabels = data.join(Xtrain_labelled_knn_pd['label'], how = 'left')

In [82]:
data_with_knnlabels.groupby('label').agg({'videoID':'count'})

,videoID
label,
0.0,2227
1.0,97
2.0,15


In [83]:
data_with_knnlabels.columns

Index(['videoID', 'channelID', 'title', 'name', 'fe_segment_duration_0',
       'fe_segment_duration_1', 'fe_segment_duration_2', 'views_0', 'views_1',
       'views_2',
       ...
       'c375', 'c376', 'c377', 'c378', 'c379', 'c380', 'c381', 'c382', 'c383',
       'label'],
      dtype='object', length=403)

In [91]:
analyse = data_with_knnlabels[['title', 'label', 'videoID', 'name']]
analyse[analyse.label == 1.0]

,title,label,videoID,name
18,How I make money using Credit Cards | Ankur Wa...,1.0,dbBbaQqIOu0,warikoo
86,Traditional Banks vs Neo Banks | Digital Banks...,1.0,TKc1nUDWaSk,warikoo
104,Why I started investing in Gold? | CA Rachana ...,1.0,PdduvP_KlcI,CARachanaRanade
105,Is an Online MBA worth it? | Ankur Warikoo,1.0,PeqgHcl5LMo,warikoo
109,The Investor Who Saved Marvel From Bankruptcy,1.0,cs7aPqw38j0,namaskarprasad
...,...,...,...,...
2007,Mamaearth IPO - The Real Truth #mamaearth #ipo,1.0,CV6Z_NFInks,namaskarprasad
2035,My Portfolio Revealed | Mutual Fund SIP Invest...,1.0,ewnac6e2hmw,PushkarRajThakurOfficial
2222,Earn 1 LAKH/month and MORE with ChatGPT! Artif...,1.0,uXRgmP7SIyk,warikoo
2227,Mutual Funds Investment | How to Choose Best M...,1.0,6fQwVxqqpAg,PushkarRajThakurOfficial


In [85]:
silhouette_score(Xtrain_pd, labels_knn)

/Users/apramod/.pyenv/versions/3.8.12/envs/sebicorps/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7328993721500769

In [86]:
# DBSCAN

dbscan_algo = DBSCAN(min_samples = 1, eps=7) #euclidean distance used here
dbscan_algo.fit(Xtrain_pd)

DBSCAN(eps=7, min_samples=1)

In [699]:
labels_dbscan = dbscan_algo.labels_.reshape((len(Xtrain_pd),1))

Xtrain_labelled_dbscan_pd = pd.DataFrame(Xtrain_pd)
Xtrain_labelled_dbscan_pd.insert(Xtrain_labelled_dbscan_pd.shape[1], 'label', labels_dbscan)

In [700]:
len(np.unique(dbscan_algo.labels_))

3

In [701]:
silhouette_score(Xtrain_pd, labels_dbscan)

/Users/apramod/.pyenv/versions/3.8.12/envs/sebicorps/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8154968900301026

In [702]:
Xtrain_labelled_dbscan_pd.groupby('label').agg({'fe_segment_duration':'count'})

,fe_segment_duration
label,
0,3666
1,46
2,37


In [839]:
import pandas as pd

In [ ]:
data = pd.read_parquet('/Users/apramod/Downloads/part-00000-026e553b-384c-4dc1-a137-a86e1eba3558-c000.snappy.parquet')